# Port ANCCR function to Python

Utilizing `calculateANCCR` function in `ANCCR.functions.calculateANCCR.m`.

In [31]:
from typing import Dict, List

import numpy as np

In [78]:
meanITI = 12
numrewards = 8  # 500

# anccr model parameters
samplingperiod = 0.2
alpha_anccr = {}
alpha_anccr["exponent"] = 0.1
alpha_anccr["init"] = 0.25
alpha_anccr["min"] = 0.02
alpha_r = 0.2
w = 0.5          
k = 1
minimumrate = 10 ** -3
maximumjitter = 0.1
beta = np.array([[1]])  # Check what beta does in calculate_anccr
threshold = 0.6
T = meanITI*10  # have very large T

nIter = 100
rwrsp = np.zeros((numrewards,nIter))

In [79]:
eventlog = np.array([
    [1.0, 2.4589, 1.0],
    [1.0, 3.6462, 1.0],
    [1, 28.4103, 1.0],
    [1, 29.4976, 1.0],
    [1, 34.9972, 1.0],
    [1, 62.9270, 1.0],
    [1, 78.2671, 1.0],
    [1, 85.5094, 1.0],
])
eventlog

array([[ 1.    ,  2.4589,  1.    ],
       [ 1.    ,  3.6462,  1.    ],
       [ 1.    , 28.4103,  1.    ],
       [ 1.    , 29.4976,  1.    ],
       [ 1.    , 34.9972,  1.    ],
       [ 1.    , 62.927 ,  1.    ],
       [ 1.    , 78.2671,  1.    ],
       [ 1.    , 85.5094,  1.    ]])

In [80]:
samplinginterval = samplingperiod
alpha = alpha_anccr

if alpha_r > 1:
    alpha_r = 1

optolog = np.zeros((eventlog.shape[0], 2))
omidx = np.array([[np.nan], [np.nan]])
exact_mean_or_not = 0
nevent_for_edge = 0

# omtrue: whether the omission state will be used or not in the calculation of ANCCR
omtrue = np.zeros((len(omidx),), dtype=bool)

uniquetime = np.unique(eventlog[:, 1])

# if more than one event happens at the same time, assume random perceptual delay between them
for jt in range(len(uniquetime)):
    if sum(eventlog[:, 1] == uniquetime[jt]) == 1:
        continue
    idx = np.where(eventlog[:, 1] == uniquetime[jt])[0]
    eventlog[idx[1:], 1] += np.random.randn(len(idx) - 1) * maximumjitter

eventlog = eventlog[eventlog[:, 1].argsort()]

ntime = eventlog.shape[0]

nstimuli = len(np.unique(eventlog[:, 0]))

samplingtime = np.arange(0, eventlog[-1, 1], samplinginterval)

# if T is a vector, use T(jt) for the calculation at time jt. otherwise, use fixed T
if isinstance(T, (int, float)):
    T = np.full((eventlog.shape[0],), T)

gamma = np.exp(-1 / T)

# Initialize model values
Eij = np.zeros((nstimuli, ntime))
Ei = np.zeros((nstimuli, ntime))
Mi = np.zeros((nstimuli, ntime))
Delta = np.zeros((nstimuli, ntime))

Mij = np.zeros((nstimuli, nstimuli, ntime))
PRC = np.zeros((nstimuli, nstimuli, ntime))
SRC = np.zeros((nstimuli, nstimuli, ntime))
NC = np.zeros((nstimuli, nstimuli, ntime))
ANCCR = np.zeros((nstimuli, nstimuli, ntime))
Rs = np.zeros((nstimuli, nstimuli, ntime))

R = np.zeros((nstimuli, nstimuli))

numevents = np.zeros((nstimuli,))

DA = np.zeros((ntime,))

beta = beta[np.unique(eventlog[:, 0]).astype(int)[0] - 1]

Imct = beta > threshold

nextt = 0
numsampling = 0

In [94]:
jt = 2
skip = False
je = int(eventlog[jt, 0]) - 1

if je in omidx[0]:
    if not omtrue[omidx[0] == je]:
        Delta[:, jt] = Delta[:, jt - 1]
        Eij[:, jt] = Eij[:, jt - 1]
        Mij[:, :, jt] = Mij[:, :, jt - 1]
        PRC[:, :, jt] = PRC[:, :, jt - 1]
        SRC[:, :, jt] = SRC[:, :, jt - 1]
        NC[:, :, jt] = NC[:, :, jt - 1]
        skip = True


numevents[je] += 1
if exact_mean_or_not == 0:
    if not isinstance(alpha, dict):
        alphat = alpha
    else:
        # if alpha is structure, alpha exponentially decreases from alpha.init to alpha.min w/ alpha.exponent decrease constant
        alphat = np.exp(-alpha['exponent'] * (jt + 1)) * (alpha['init'] - alpha['min']) + alpha['min']
else:
    alphat = 1 / numevents[je]

print("alphat = 0.2281")
print(round(alphat, 4))

alphat = 0.2281
0.1904


In [95]:
if jt > 0:
    # update delta w/prev value
    Delta[:, jt] = Delta[:, jt - 1] * gamma[jt] ** (eventlog[jt, 1] - eventlog[jt - 1, 1])
    # update instantaneous elig. trace w/prev value
    Eij[:, jt] = Eij[:, jt - 1] * gamma[jt] ** (eventlog[jt, 1] - eventlog[jt - 1, 1])
    # update average elig. trace w/prev value
    Mij[:, :, jt] = Mij[:, :, jt - 1]
    # update anccr w/prev value
    ANCCR[~np.isin(np.arange(nstimuli), je), :, jt] = ANCCR[~np.isin(np.arange(nstimuli), je), :, jt - 1]

# Indicator for whether event has recently happened
# Delta resets to one at every instance of event w/o cumulative sum
Delta[je, jt] = 1
# Increment inst. elig. trace by 1 for event that occurred
Eij[je, jt] += 1
# Update avg. elig. trace
Mij[:, je, jt] += alphat * (Eij[:, jt] - Mij[:, je, jt]) * Imct[je]

# Subtract baseline elig. from avg. elig. to find successor rep.
PRC[:, :, jt] = Mij[:, :, jt] - np.tile(Mi[:, jt], (nstimuli, 1)).T
# Calculate predecessor rep from successor rep.
SRC[:, :, jt] = PRC[:, :, jt] * np.tile(Mi[:, jt], (nstimuli, 1)) / np.tile(Mi[:, jt], (nstimuli, 1)).T

print(Delta)
print(Eij)
print(Mij)
print(PRC)
print(SRC)

[[1. 1. 1. 0. 0. 0. 0. 0.]]
[[1.         1.99015462 2.61905854 0.         0.         0.
  0.         0.        ]]
[[[0.22811261 0.59516018 0.98048653 0.         0.         0.
   0.         0.        ]]]
[[[ 0.22811261 -0.18818689 -0.64904152  0.          0.
    0.          0.          0.        ]]]
[[[ 0.         -0.18818689 -0.64904152  0.          0.
    0.          0.          0.        ]]]


In [96]:
# Zero out values that may approach -Inf
belowminrate = Mi[:, jt] / T[jt] < minimumrate
SRC[belowminrate, :, jt] = 0

print(belowminrate)
print(SRC)

[False]
[[[ 0.         -0.18818689 -0.64904152  0.          0.
    0.          0.          0.        ]]]


In [97]:
# something to make sure only calculating contingency and R after experiencing first outcome
PRC[numevents == 0, :, jt] = 0
PRC[:, numevents == 0, jt] = 0
SRC[numevents == 0, :, jt] = 0
SRC[:, numevents == 0, jt] = 0
R[:, numevents == 0] = 0
R[numevents == 0, :] = 0

# Calculate net contingency, weighted sum of SRC/PRC
NC[:, :, jt] = w * SRC[:, :, jt] + (1 - w) * PRC[:, :, jt]

# Indicator for whether an event is associated with another event
Iedge = np.mean(NC[:, je, max([0, jt - nevent_for_edge]):jt+1], axis=1) > threshold
Iedge[je] = False

# once the cause of reward state is revealed, omission state of that reward state can be used for calculation of ANCCR. Before that, omission state is ignored
if je in omidx[1] and np.sum(Iedge) > 0:
    omtrue[omidx[1] == je] |= True

# calculate ANCCR for every event
# Rjj is externally driven; the magnitude of stimulus an animal just experienced
R[je, je] = eventlog[jt, 2]

for ke in range(nstimuli):
    # Update edge indicator
    Iedge_ke = np.mean(NC[:, ke, max([0, jt - nevent_for_edge]):jt+1], axis=1) > threshold
    print(Iedge_ke)
    Iedge_ke[ke] = False
    # update ANCCR
    ANCCR[ke, :, jt] = NC[ke, :, jt] * R[ke, :] - np.sum(ANCCR[:, :, jt] * Delta[:, jt].reshape(-1,1) * np.tile(Iedge_ke.reshape(-1,1), (1,nstimuli)), axis=0)
    
print(Iedge_ke)
print(ANCCR)

[False]
[False]
[[[ 0.1140563  -0.18818689 -0.64904152  0.          0.
    0.          0.          0.        ]]]


In [98]:
if not (optolog[jt, 0] == 1): # If target is not inhibited, normal DA
    DA[jt] = np.sum(ANCCR[je,:,jt]*Imct)
else: # If target is inhibited, replace DA
    DA[jt] = optolog[jt,1]

if je in omidx[0]:
    je_om = np.where(je == omidx[0])[0]
    # if the current state is omission of j, R(omission,j) = R(j,j)
    R[je, omidx[je_om, 1]] = R[omidx[je_om, 1], omidx[je_om, 1]]
    # omission state is an MCT
    Imct[je] = True

# This must come after opto s.t. Imct is not formed before opto applied
Imct[je] |= DA[jt] + beta[je] > threshold

# Update estimated reward value
Rs[:, :, jt] = R
if DA[jt] >= 0:
    # For positive DA response, use standard update rule
    R[:, je] += alpha_r * (eventlog[jt, 2] - R[:, je])
else:
    # For negative DA response, use overprediction update rule
    if np.any(Iedge):
        R[Iedge, je] -= alpha_r * R[Iedge, je] * ((Delta[Iedge, jt] / numevents[Iedge]) / np.sum((Delta[Iedge, jt] / numevents[Iedge])))
    else:
        R[:, je] = R[:, je]
        
print(DA)
print(R)
print(Rs)

[ 0.1140563  -0.18818689 -0.64904152  0.          0.          0.
  0.          0.        ]
[[1.]]
[[[1. 1. 1. 0. 0. 0. 0. 0.]]]


In [99]:
# Update sample eligibility trace (Mi-)
if jt < ntime:
    # Time to sample baseline b/t events
    subsamplingtime = samplingtime[(samplingtime >= eventlog[jt, 1]) & (samplingtime < eventlog[jt+1, 1])]

    Ei[:, jt+1] = Ei[:, jt] * gamma[jt] ** samplinginterval
    if subsamplingtime.size > 0:
        for jjt in range(nextt, jt + 1):
            if eventlog[jjt, 0] in omidx[0]:
                if not omtrue[omidx[0] == eventlog[jjt, 0]]:
                    continue
            Ei[int(eventlog[jjt, 0]) - 1, jt+1] += gamma[jt] ** (subsamplingtime[0] - eventlog[jjt, 1])
        nextt = jt + 1

    # update alpha of sample eligibility trace
    if exact_mean_or_not == 0:
        if not isinstance(alpha, dict):
            alphat = alpha
        else:
            alphat = np.exp(-alpha['exponent'] * (jt + 1)) * (alpha['init'] - alpha['min']) + alpha['min']
    else:
        alphat = 1 / (numsampling + 1)

    # Update avg. sample eligibility trace
    Mi[:, jt+1] = Mi[:, jt] + k * alphat * (Ei[:, jt+1] - Mi[:, jt])
    for iit in range(1, len(subsamplingtime)):
        iit += 1
        if exact_mean_or_not == 0:
            if not isinstance(alpha, dict):
                alphat = alpha
            else:
                alphat = np.exp(-alpha['exponent'] * (jt + 1)) * (alpha['init'] - alpha['min']) + alpha['min']
        else:
            alphat = 1 / (numsampling + iit)

        Ei[:, jt+1] *= gamma[jt] ** samplinginterval
        Mi[:, jt+1] += k * alphat * (Ei[:, jt+1] - Mi[:, jt+1])
    numsampling += len(subsamplingtime)
    
print(alphat)
print(Ei)
print(Mi)
print(numsampling)

0.1903881907567951
[[0.         0.9905359  1.61919752 2.59754669 0.         0.
  0.         0.        ]]
[[0.         0.78334708 1.62952805 2.26531442 0.         0.
  0.         0.        ]]
135


In [100]:
DA

array([ 0.1140563 , -0.18818689, -0.64904152,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ])

In [16]:
for jjt in range(0, jt+1):
    print(jjt)

0


In [77]:
int(eventlog[jjt, 0])

1

In [21]:
Ei[int(eventlog[jjt, 0])-1, jt+1]

0.0

In [19]:
eventlog

array([[ 1.    ,  2.4589,  1.    ],
       [ 1.    ,  3.6462,  1.    ],
       [ 1.    , 28.4103,  1.    ],
       [ 1.    , 29.4976,  1.    ],
       [ 1.    , 34.9972,  1.    ],
       [ 1.    , 62.927 ,  1.    ],
       [ 1.    , 78.2671,  1.    ],
       [ 1.    , 85.5094,  1.    ]])

In [22]:
gamma[jt] ** (subsamplingtime[0] - eventlog[jjt, 1])

0.9988248576878122

In [23]:
subsamplingtime

array([2.6, 2.8, 3. , 3.2, 3.4, 3.6])

In [24]:
eventlog[jjt, 1]

2.4589

In [25]:
eventlog

array([[ 1.    ,  2.4589,  1.    ],
       [ 1.    ,  3.6462,  1.    ],
       [ 1.    , 28.4103,  1.    ],
       [ 1.    , 29.4976,  1.    ],
       [ 1.    , 34.9972,  1.    ],
       [ 1.    , 62.927 ,  1.    ],
       [ 1.    , 78.2671,  1.    ],
       [ 1.    , 85.5094,  1.    ]])

In [ ]:
Ei[int(eventlog[jjt, 0]), jt+1] += gamma[jt] ** (subsamplingtime[0] - eventlog[jjt, 1])

In [10]:
gamma[jt]

0.991701292638876

In [11]:
samplinginterval

0.2

In [15]:
gamma[jt] ** samplinginterval

0.9983347214509387

In [12]:
Ei

array([[0., 0., 0., 0., 0., 0., 0., 0.]])